# Building a spam filter with Naive Bayes

We'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). You can also download the dataset directly from [this link](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection). The data collection process is described in more details on [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the authors' papers.

In [1]:
import pandas as pd

sms = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])
sms.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [2]:
sms.describe()

,Label,SMS
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [3]:
def pc_spam(labels):
    total = labels.size
    counts = labels.value_counts()
    return (counts['spam'] / total) * 100

In [4]:
print('{0:.2f}% of {1} messages are marked as Spam'.format(pc_spam(sms['Label']), len(sms)))

13.41% of 5572 messages are marked as Spam


To test the spam filter, we're first going to split our dataset into two categories:

1. A **training set**, which we'll use to "train" the computer how to classify messages.
2. A **test set**, which we'll use to test how good the spam filter is with classifying new messages.

In [5]:
df = sms.sample(frac=1, random_state=1)
lim = round(.8 * len(df))
train = df.iloc[:lim].copy()
test = df.iloc[lim:].copy()

print('{0:.2f}% of {1} training messages are marked as Spam'.format(pc_spam(train['Label']), len(train)))
print('{0:.2f}% of {1} test messages are marked as Spam'.format(pc_spam(test['Label']), len(test)))

13.46% of 4458 training messages are marked as Spam
13.20% of 1114 test messages are marked as Spam


## Data cleaning

To calculate all the necessary probabilities for the spam filter, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need. 

In [6]:
train.head()

,Label,SMS
1078,ham,"Yep, by the pretty sculpture"
4028,ham,"Yes, princess. Are you going to make me moan?"
958,ham,Welp apparently he retired
4642,ham,Havent.
4674,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [7]:
import re
def clean(string):
    string = re.sub('\W', ' ', string).lower()
    return re.sub(' +', ' ', string).split()

train['SMS'] = train['SMS'].apply(clean)
train.head()

,Label,SMS
1078,ham,"[yep, by, the, pretty, sculpture]"
4028,ham,"[yes, princess, are, you, going, to, make, me,..."
958,ham,"[welp, apparently, he, retired]"
4642,ham,[havent]
4674,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,..."


Next we will create a list with all of the unique words that occur in the messages of our training set. This will be our **vocabulary**. 

In [8]:
vocabulary = []
for msg in train['SMS']:
    for word in msg:
        vocabulary.append( word )
        
vocabulary = list(set(vocabulary))
print('{} unique words identified'.format(len(vocabulary)))

7783 unique words identified


## Creating a dictionary

We will use the created vocabulary to generate a dictionary which counts the occurence of each unique word for every message. This will be provide the basis for calculating the probabilities required to build our spam filter.

In [9]:
word_counts_per_sms = {unique_word: [0] * len(train) for unique_word in vocabulary}

for index, sms in enumerate(train['SMS']): 
    for word in sms:
        word_counts_per_sms[word][index] += 1
        
word_counts = pd.DataFrame(word_counts_per_sms)
train = pd.concat([train, word_counts], axis=1, ignore_index=False)
train = train[pd.notnull(train['Label'])]
train.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[go, until, jurong, point, crazy, available, o...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ham,"[ok, lar, joking, wif, u, oni]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ham,"[u, dun, say, so, early, hor, u, c, already, t...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ham,"[nah, i, don, t, think, he, goes, to, usf, he,...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
5,spam,"[freemsg, hey, there, darling, it, s, been, 3,...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Calculating constants

With this word dictionary, we can begin creating the spam filter. The Naive Bayes algorithm will need to know the probability values of the two equations below to be able to classify new messages:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
\end{equation}

\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

To calculate P($w_i$|Spam) and P($w_i$|Ham) inside the formulas above, we need to use these equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

We will start by calculating some of the constants required.

In [10]:
p_spam = pc_spam(train['Label']) / 100
p_ham = 1 - p_spam
print('P(Spam) = {0:.2f}'.format(p_spam))
print('P(Ham) = {0:.2f}'.format(p_ham))

P(Spam) = 0.13
P(Ham) = 0.87


In [12]:
n_vocab = len(vocabulary)

spam_messages = train[train['Label'] == 'spam']
spam_word_count = spam_messages['SMS'].apply(len)
n_spam = spam_word_count.sum()

ham_messages = train[train['Label'] == 'ham']
ham_word_count = ham_messages['SMS'].apply(len)
n_ham = ham_word_count.sum()

alpha = 1

print('N_spam = {}'.format(n_spam))
print('N_ham = {}'.format(n_ham))
print('N_vocab = {}'.format(n_vocab))
print('Alpha = {}'.format(alpha))

N_spam = 15190
N_ham = 57237
N_vocab = 7783
Alpha = 1


## Calculating parameters

With the constants calculated, we can now focus on the parameters which define the Naive Bayes filter. These are the probabilities of each word occuring with the given outcomes. With each new message these parameters are adjusted. A reminder of the equations we are using: 

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

In [13]:
spam_params = {unique_word:0 for unique_word in vocabulary}
ham_params = {unique_word:0 for unique_word in vocabulary}

for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocab)
    spam_params[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocab)
    ham_params[word] = p_word_given_ham

## Classifying a new message

Now that we've calculated all the constants and parameters we need, we can start creating the spam filter.

In [17]:
def classify(message):
    # cleaning the message
    message = re.sub('\W', ' ', message)
    message = re.sub(' +', ' ', message)
    message = message.lower().split()
    
    # initialising the probabilities
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham   
    print(message)
    
    for word in message:
        if word in spam_params:
            p_spam_given_message *= spam_params[word]
        if word in ham_params:
            p_ham_given_message *= ham_params[word]

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [18]:
msg1 = 'WINNER!! This is the secret code to unlock the money: C3421.'
msg2 = 'Sounds good, Tom, then see u there'

print('--- Test 1 ---')
classify(msg1)

print('\n--- Test 2 ---')
classify(msg2)

--- Test 1 ---
['winner', 'this', 'is', 'the', 'secret', 'code', 'to', 'unlock', 'the', 'money', 'c3421']
P(Spam|message): 4.9813712674139326e-29
P(Ham|message): 1.538857970697398e-25
Label: Ham

--- Test 2 ---
['sounds', 'good', 'tom', 'then', 'see', 'u', 'there']
P(Spam|message): 2.9292764222612153e-24
P(Ham|message): 5.136710135587318e-22
Label: Ham


## Measuring accuracy

Now we managed to create a spam filter, and classified two new messages. We'll now try to determine how well the spam filter does on our test set of 1,114 messages.

The algorithm will output a classification label for every message in our test set, which we'll be able to compare with the actual label (given by a human). Note that, in training, our algorithm didn't see these 1,114 messages, so every message in the test set is practically new from the perspective of the algorithm.

In [21]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in spam_params:
            p_spam_given_message *= spam_params[word]

        if word in spam_params:
            p_ham_given_message *= ham_params[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [22]:
test['predicted'] = test['SMS'].apply(classify_test_set)
test.head()

,Label,SMS,predicted
2131,ham,Later i guess. I needa do mcat study too.,ham
3418,ham,But i haf enuff space got like 4 mb...,ham
3424,spam,Had your mobile 10 mths? Update to latest Oran...,ham
1538,ham,All sounds good. Fingers . Makes it difficult ...,ham
5393,ham,"All done, all handed in. Don't know if mega sh...",ham


Now we can compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages. To make the measurement, we'll use accuracy as a metric:

\begin{equation}
\text{Accuracy} = \frac{\text{number of correctly classified messages}}{\text{total number of classified messages}}
\end{equation}

In [26]:
correct = 0
total = len(test)
for row in test.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
        
print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 965
Incorrect: 149
Accuracy: 0.8662477558348295


## Next steps...

Here are some potential next steps:

- Isolate the messages that were classified incorrectly and try to figure out why the algorithm reached the wrong conclusions.
- Make the filtering process more complex by making the algorithm sensitive to letter case.